# Exporting geometry data for a ChemProp3D model for UVVis peaks

In [1]:
%load_ext autoreload
%autoreload 2
    
import os
import django


os.environ["DJANGOCHEMDIR"]="djangochem.settings.orgel"
os.environ["DJANGO_SETTINGS_MODULE"]="djangochem.settings.orgel"
django.setup()

from pgmols.models import Species, Geom, Calc
from jobs.models import Job, JobConfig

In [2]:
from nff.utils.misc import read_csv

direc = '/home/saxelrod/chemdata/fluodye'
name = '20211220_deep4chem_chemfluor_dcm_chemprop3d.csv'
csv_path = os.path.join(direc, name)

dic = read_csv(csv_path)


In [4]:
# from tqdm import tqdm

# method_name = 'dft_d3_gga_bp86'
# method_descrip = 'Orca bp86/def2-SVP D3 DFT RI'
# config_name = 'bp86_d3_def2svp_opt_orca'

# conformers = []

# for smiles in tqdm(dic['smiles']):
#     spec = Species.objects.get(smiles=smiles, 
#                                group__name='fluodye')
#     geoms = spec.geom_set.filter(converged=True,
#                                  parentjob__config__name=config_name)
# #     if not geoms:
# #         continue
        
#     calc_pks = list(geoms.values_list('calcs', flat=True))
#     calcs = Calc.objects.filter(pk__in=calc_pks,
#                                 method__name=method_name,
#                                 method__description=method_descrip,
#                                 props__totalenergy__isnull=False)
    
#     lowest_en_calc = calcs.order_by("props__totalenergy").first()
#     lowest_en_geom = lowest_en_calc.geoms.get()
    
    
#     # get the bonded neighbor list
#     rd_mol = lowest_en_geom.as_rdkit_mol()
#     conformers.append([{"rd_mol": rd_mol, "geom_id": lowest_en_geom.id}])
    

#### Save summary file and pickles, which is needed for generating splits and making the dataset in CP3D

In [5]:
# from rdkit import Chem
# import pickle
# import json

# rd_folder = 'rd_mols'
# rd_path = os.path.join(dset_dir, rd_folder)

# summary_dic = {}

# for i, confs in enumerate(conformers):
#     sub_dic = {key: val[i] for key, val in dic.items()}
#     confs = [{**conf_dic, "boltzmannweight": 1.0}
#             for conf_dic in confs]
#     pick_dic = {"conformers": confs,
#                 **sub_dic}
#     smiles = pick_dic['smiles']
    
#     inchikey = Chem.InchiToInchiKey(Chem.MolToInchi(Chem.MolFromSmiles(smiles),
#                                                     options=" -RecMet  -FixedH "))
#     rel_path = os.path.join(rd_folder, f"{inchikey}.pickle")
#     abs_path = os.path.join(dset_dir, rel_path)
    
#     summary_dic[smiles] = {key: val for key, val in pick_dic.items()
#                           if key != 'conformers'}
#     summary_dic[smiles].update({"pickle_path": rel_path})
    
#     if not os.path.isdir(rd_path):
#         os.makedirs(rd_path)
        
#     with open(abs_path, 'wb') as f:
#         pickle.dump(pick_dic, f)

# summ_path = os.path.join(dset_dir, 'summary.json')
# with open(summ_path, 'w') as f:
#     json.dump(summary_dic, f, indent=4)

Now run `scripts/cp3d/make_dset/make_dset.sh`, with the right config files specified

In [7]:
# from matplotlib import pyplot as plt
# import numpy as np

# lam = dset.props['peakwavs_max'].reshape(-1).numpy()
# plt.hist(lam)
# plt.show()

# mae_lam = np.mean(abs(lam - np.mean(lam)))
# print(mae_lam)

In [8]:
# from nff.data import Dataset

# test_path = ("/home/saxelrod/Repo/projects/master/NeuralForceField/tutorials/data"
#              "/uvvis_cp3d/ndu/0/test.pth.tar")
# test = Dataset.from_file(test_path)

In [9]:
# batch = next(iter(test))

In [ ]:
# batch

In [ ]:
# dic['peakwavs_max'][dic['smiles'].index(batch['smiles'])]

In [ ]:
# from ase import Atoms
# import nglview as nv

# nxyz = batch['nxyz'].numpy()
# atoms = Atoms(numbers=nxyz[:, 0].astype('int'),
#               positions=nxyz[:, 1:])
# display(nv.show_ase(atoms))

# Train a PaiNN model

In [10]:
import sys
from pathlib import Path

# change to your NFF path
sys.path.insert(0, "/home/saxelrod/Repo/projects/master/NeuralForceField")

import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler


from nff.data import Dataset, split_train_validation_test, collate_dicts, to_tensor
from nff.train import Trainer, get_trainer, get_model, load_model, loss, hooks, metrics, evaluate


In [15]:
DEVICE = 0

dset_dir = '/home/saxelrod/Repo/projects/master/NeuralForceField/tutorials/data/uvvis_cp3d/cp3d'

OUTDIR = './painn_uvvis_att_aggr'
# batch size used in the original paper
BATCH_SIZE = 10

if os.path.exists(OUTDIR):
    newpath = os.path.join(os.path.dirname(OUTDIR), 'backup')
    if os.path.exists(newpath):
        shutil.rmtree(newpath)
        
    shutil.move(OUTDIR, newpath)
    
out = []
for name in ['train', 'val', 'test']:
    path = os.path.join(dset_dir, '0', f'{name}.pth.tar')
    out.append(Dataset.from_file(path))
train, val, test = out

In [16]:
modelparams = {"feat_dim": 128,
              "activation": "swish",
              "n_rbf": 20,
              "cutoff": 5.0,
              "num_conv": 3,
              "output_keys": ["peakwavs_max"],
              "grad_keys": [],
              "skip_connection": {"peakwavs_max": False},
              "learnable_k": False,
              "conv_dropout": 0.0,
              "readout_dropout": 0.0,
              "pool_dic": {"peakwavs_max": {"name": "attention",
                                            "param": {"prob_func": "softmax",
                                                      "feat_dim": 128,
                                                      "att_act": "swish",
                                                      "mol_fp_act": "swish",
                                                      "num_out_layers": 2,
                                                      "out_dim": 1}}}
               
#               "pool_dic": {"peakwavs_max": {"name": "mean",
#                                             "param": {}}}
              }
model = get_model(modelparams, model_type="Painn")



In [17]:
BATCH_SIZE = 10

train_loader = DataLoader(train, 
                          batch_size=BATCH_SIZE, 
                          collate_fn=collate_dicts,
                          sampler=RandomSampler(train))

val_loader = DataLoader(val, batch_size=BATCH_SIZE, collate_fn=collate_dicts)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, collate_fn=collate_dicts)

# loss trade-off used in the original paper
loss_fn = loss.build_mse_loss(loss_coef={'peakwavs_max': 1})

trainable_params = filter(lambda p: p.requires_grad, model.parameters())

# learning rate used in the original paper
optimizer = Adam(trainable_params, lr=1e-4)


train_metrics = [
    metrics.MeanAbsoluteError('peakwavs_max')
]


train_hooks = [
    hooks.MaxEpochHook(1000),
    hooks.CSVHook(
        OUTDIR,
        metrics=train_metrics,
    ),
    hooks.PrintingHook(
        OUTDIR,
        metrics=train_metrics,
        separator = ' | ',
        time_strf='%M:%S'
    ),
    hooks.ReduceLROnPlateauHook(
        optimizer=optimizer,
        # patience in the original paper
        patience=50,
        factor=0.5,
        min_lr=1e-7,
        window_length=1,
        stop_after_min=True
    )
]

T = Trainer(
    model_path=OUTDIR,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_loader=train_loader,
    validation_loader=val_loader,
    checkpoint_interval=1,
    hooks=train_hooks,
    mini_batches=1
)

In [ ]:
T.train(device=DEVICE, n_epochs=1000)

 Time | Epoch | Learning rate | Train loss | Validation loss | MAE_peakwavs_max | GPU Memory (MB)


100%|███████████████████████████████████████████████████████████████████████████████████████████████▌| 231/232 [00:09<00:00, 23.92it/s]


08:29 |     1 |     1.000e-04 | 46043.2406 |      10465.1701 |          72.4960 |            1038


100%|███████████████████████████████████████████████████████████████████████████████████████████████▌| 231/232 [00:08<00:00, 26.54it/s]


08:38 |     2 |     1.000e-04 | 10337.0465 |      10474.1699 |          69.7394 |            1038


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████▌| 231/232 [00:20<00:00, 11.35it/s]


30:24 |   115 |     1.000e-04 |   623.5915 |       2113.8618 |          31.3109 |            1041


100%|███████████████████████████████████████████████████████████████████████████████████████████████▌| 231/232 [00:10<00:00, 21.16it/s]


30:36 |   116 |     1.000e-04 |   467.8991 |       2339.3939 |          32.1726 |            1041


 58%|███████████████████████████████████████████████████████▊                                        | 135/232 [00:05<00:04, 22.18it/s]

In [ ]:
results, targets, val_loss = evaluate(T.get_best_model(), 
                                      test_loader, 
                                      loss_fn, 
                                      device=DEVICE)

units = {
    'peakwavs_max': 'nm'
}

fig, ax_fig = plt.subplots(1, 2, figsize=(12, 6))

for ax, key in zip(ax_fig, units.keys()):
    pred_fn = torch.cat
    targ_fn = torch.cat
    if all([len(i.shape) == 0 for i in results[key]]):
        pred_fn = torch.stack
    if all([len(i.shape) == 0 for i in targets[key]]):
        targ_fn = torch.stack
        
    pred = pred_fn(results[key], dim=0).view(-1).detach().cpu().numpy()
    targ = targ_fn(targets[key], dim=0).view(-1).detach().cpu().numpy()

    mae = abs(pred-targ).mean()
    
    ax.hexbin(pred, targ, mincnt=1)
    
    lim_min = min(np.min(pred), np.min(targ)) * 1.1
    lim_max = max(np.max(pred), np.max(targ)) * 1.1
    
    ax.set_xlim(lim_min, lim_max)
    ax.set_ylim(lim_min, lim_max)
    ax.set_aspect('equal')
    
    ax.plot((lim_min, lim_max),
            (lim_min, lim_max),
            color='#000000',
            zorder=-1,
            linewidth=0.5)
    
    ax.set_title(key.upper(), fontsize=14)
    ax.set_xlabel('predicted %s (%s)' % (key, units[key]), fontsize=12)
    ax.set_ylabel('target %s (%s)' % (key, units[key]), fontsize=12)
    ax.text(0.1, 0.9, 'MAE: %.2f %s' % (mae, units[key]), 
           transform=ax.transAxes, fontsize=14)

plt.show()